Розробити  програмний  скрипт,  що  реалізує  багатокритеріальне  оцінювання  ефективності 
**ноутбуків**.  Формування  показників  та  критеріїв  ефективності,  синтез 
багатокритеріальної оптимізаційної моделі здійснити самостійно.

In [68]:
import pandas as pd 
import numpy as np
import re
from itertools import groupby
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt

In [69]:
df = pd.read_csv('Cleaned_Laptop_data.csv')
df

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
0,Lenovo,A6-9225,AMD,A6-9225 Processor,10th,4 GB GB,DDR4,0 GB,1024 GB,Windows,...,Missing,0,No,No,24990,32790,23,3.7,63,12
1,Lenovo,Ideapad,AMD,APU Dual,10th,4 GB GB,DDR4,0 GB,512 GB,Windows,...,Missing,0,No,No,19590,21325,8,3.6,1894,256
2,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,...,Missing,0,No,No,19990,27990,28,3.7,1153,159
3,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,...,Missing,0,No,No,21490,27990,23,3.7,1153,159
4,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,256 GB,0 GB,Windows,...,Missing,0,No,No,24990,33490,25,3.7,1657,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,ASUS,ROG,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,...,16,1,No,No,144990,194990,25,0.0,0,0
892,ASUS,Ryzen,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,...,Missing,1,No,No,149990,0,0,0.0,0,0
893,ASUS,Ryzen,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,...,16,1,No,No,142990,0,0,0.0,0,0
894,SAMSUNG,Galaxy,Qualcomm,Snapdragon 7c,Missing,4 GB GB,LPDDR4X,0 GB,512 GB,Windows,...,Missing,0,No,No,38990,47990,18,0.0,0,0


In [70]:
df.dtypes

brand               object
model               object
processor_brand     object
processor_name      object
processor_gnrtn     object
ram_gb              object
ram_type            object
ssd                 object
hdd                 object
os                  object
os_bit              object
graphic_card_gb      int64
weight              object
display_size        object
warranty             int64
Touchscreen         object
msoffice            object
latest_price         int64
old_price            int64
discount             int64
star_rating        float64
ratings              int64
reviews              int64
dtype: object

In [71]:
cleaned_df = df.copy()

cleaned_df = cleaned_df.drop(cleaned_df[cleaned_df.eq('Missing').any(axis=1)].index)
cleaned_df[['ram_gb', 'ssd', 'hdd']] = cleaned_df[['ram_gb', 'ssd', 'hdd']].replace('GB', '', regex=True).astype(int)

# cleaned_df = cleaned_df[cleaned_df['processor_gnrtn'] != 'Missing']
cleaned_df['processor_gnrtn'] = cleaned_df['processor_gnrtn'].str.replace('th', '').astype(int)
cleaned_df['os'] = cleaned_df['os'].replace({'Windows': 1, 'Mac': 1, 'DOS': 0})
cleaned_df['os_bit'] = cleaned_df['os_bit'].replace('-bit', '', regex=True).astype(int)
cleaned_df['Touchscreen'] = cleaned_df['Touchscreen'].replace({'Yes': 1, 'No': 0})
cleaned_df['msoffice'] = cleaned_df['msoffice'].replace({'Yes': 1, 'No': 0})
cleaned_df['display_size']= cleaned_df['display_size'].astype(float)

In [72]:
ram_types = df['ram_type'].unique()
sorted_ram_types = sorted(ram_types, key=lambda x: int(''.join(filter(str.isdigit, x))))

split_ram_types = [re.split(r'(\d+)', ram_type) for ram_type in sorted_ram_types]
grouped_ram_types = [list(group) for key, group in groupby(split_ram_types, key=lambda x: int(x[1]))]

for i in range(len(grouped_ram_types)):
    for j in range(len(grouped_ram_types[i])):
        if 'LP' in grouped_ram_types[i][j][0]:
            grouped_ram_types[i].insert(0, grouped_ram_types[i].pop(j))

ram_variations = [ ''.join(item) for sublist in grouped_ram_types for item in sublist]
weight_variations = list(cleaned_df['weight'].unique())
weight_variations.reverse()

ram_variations_indices = {ram_variations[i]: i for i in range(len(ram_variations))}
weight_variations_indices = {weight_variations[i]: i for i in range(len(weight_variations))}

cleaned_df['ram_type'] = cleaned_df['ram_type'].replace(ram_variations_indices)
cleaned_df['weight'] = cleaned_df['weight'].replace(weight_variations_indices)

In [73]:
cleaned_df

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
5,Avita,PURA,AMD,APU Dual,10,8,4,256,0,1,...,14.0,0,0,0,24990,33490,25,3.7,1657,234
6,HP,APU,AMD,APU Dual,10,4,4,0,1024,1,...,14.0,0,0,0,20900,22825,8,3.9,1185,141
8,HP,Athlon,AMD,Athlon Dual,10,32,4,32,0,1,...,14.0,0,0,0,26899,27668,2,0.0,0,0
10,ASUS,ExpertBook,Intel,Core i3,10,4,4,0,1024,1,...,15.6,0,0,0,34649,41990,17,2.3,3,0
12,Lenovo,v15,Intel,Core i3,10,4,4,0,1024,1,...,15.6,0,0,0,39999,43999,9,3.1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,APPLE,MacBook,M1,M1,10,16,4,1024,0,1,...,16.2,1,0,0,243990,259900,6,4.8,42,8
677,APPLE,MacBook,M1,M1,10,32,4,1024,0,1,...,16.2,1,0,0,309990,329900,6,5.0,3,1
678,APPLE,MacBook,M1,M1,10,16,4,1024,0,1,...,14.2,1,0,0,225990,239900,5,4.8,42,8
679,DELL,G7,Intel,GeForce RTX,10,8,2,0,512,0,...,15.6,0,0,0,179990,227250,20,4.5,17,2


Let's do the normalisation of the columns

In [74]:
numeric_columns = cleaned_df.select_dtypes(include='number').columns.tolist()

for col in numeric_columns:
    val = cleaned_df[col]
    cleaned_df[col] = (val - val.min()) / (val.max() - val.min())

In [75]:
weights = {
    'processor_gnrtn':1,
    'ram_gb':1,
    'ram_type':2,
    'ssd':2,
    'hdd':1,
    'os':1,
    'os_bit':2,
    'graphic_card_gb':4,
    'weight':3,
    'display_size':3,
    'warranty':2,
    'Touchscreen':0.5,
    'msoffice':1,
    'latest_price':-5,
    'old_price':2,
    'discount':3,
    'star_rating':4,
    'ratings':4,
    'reviews':2
}

In [76]:
cleaned_df['score'] = np.zeros(len(cleaned_df))
for col in numeric_columns:
    cleaned_df['score'] += cleaned_df[col] * weights[col]   

In [77]:
cleaned_df

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews,score
5,Avita,PURA,AMD,APU Dual,0.75,0.142857,1.0,0.125000,0.00,1.0,...,0.000000,0.0,0.0,0.014148,0.095716,0.438596,0.74,0.246394,0.230769,16.045277
6,HP,APU,AMD,APU Dual,0.75,0.000000,1.0,0.000000,0.50,1.0,...,0.000000,0.0,0.0,0.000000,0.065235,0.140351,0.78,0.176208,0.139053,11.463285
8,HP,Athlon,AMD,Athlon Dual,0.75,1.000000,1.0,0.015625,0.00,1.0,...,0.000000,0.0,0.0,0.020751,0.079076,0.035088,0.00,0.000000,0.000000,9.499733
10,ASUS,ExpertBook,Intel,Core i3,0.75,0.000000,1.0,0.000000,0.50,1.0,...,0.000000,0.0,0.0,0.047560,0.120009,0.298246,0.46,0.000446,0.000000,12.488742
12,Lenovo,v15,Intel,Core i3,0.75,0.000000,1.0,0.000000,0.50,1.0,...,0.000000,0.0,0.0,0.066066,0.125751,0.157895,0.62,0.001190,0.000986,12.631587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,APPLE,MacBook,M1,M1,0.75,0.428571,1.0,0.500000,0.00,1.0,...,0.333333,0.0,0.0,0.771697,0.742805,0.105263,0.96,0.006245,0.007890,13.521852
677,APPLE,MacBook,M1,M1,0.75,1.000000,1.0,0.500000,0.00,1.0,...,0.333333,0.0,0.0,1.000000,0.942868,0.105263,1.00,0.000446,0.000986,13.474890
678,APPLE,MacBook,M1,M1,0.75,0.428571,1.0,0.500000,0.00,1.0,...,0.333333,0.0,0.0,0.709433,0.685644,0.087719,0.96,0.006245,0.007890,12.489750
679,DELL,G7,Intel,GeForce RTX,0.75,0.142857,0.5,0.000000,0.25,0.0,...,0.000000,0.0,0.0,0.550313,0.649490,0.350877,0.90,0.002528,0.001972,12.856959


In [78]:
df['score'] = cleaned_df['score']

In [79]:
df.sort_values(by='score',ascending=False).head(10)

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews,score
44,realme,Book,Intel,Core i3,11th,8 GB GB,DDR4,256 GB,0 GB,Windows,...,1,No,Yes,46990,54999,14,4.4,6352,1014,22.016466
152,HP,15s,Intel,Core i3,11th,8 GB GB,DDR4,0 GB,1024 GB,Windows,...,1,No,Yes,43690,48933,10,4.1,4999,490,20.941229
323,Lenovo,Thinkbook,Intel,Core i5,11th,8 GB GB,DDR4,512 GB,0 GB,Windows,...,3,Yes,Yes,68990,159180,56,4.6,44,8,20.625608
593,MSI,Pulse,Intel,Core i7,11th,16 GB GB,DDR4,1024 GB,0 GB,Windows,...,2,No,No,129990,179990,27,4.2,14,3,20.434551
575,MSI,GF65,Intel,Core i7,10th,16 GB GB,DDR4,1024 GB,0 GB,Windows,...,2,No,No,84990,123990,31,4.4,498,64,20.186183
148,Lenovo,IdeaPad,Intel,Core i3,10th,8 GB GB,DDR4,0 GB,1024 GB,Windows,...,1,No,Yes,35990,57290,37,4.0,2124,233,19.996287
130,Lenovo,IdeaPad,Intel,Core i3,11th,8 GB GB,DDR4,512 GB,0 GB,Windows,...,2,No,Yes,42990,63790,32,4.4,984,99,19.943514
267,ASUS,TUF,Intel,Core i5,10th,8 GB GB,DDR4,512 GB,0 GB,Windows,...,1,No,Yes,65990,89990,26,4.4,3850,378,19.717999
574,MSI,GF63,Intel,Core i7,9th,8 GB GB,DDR4,512 GB,0 GB,Windows,...,2,No,No,66990,104990,36,4.5,1639,230,19.702421
624,acer,Predator,Intel,Core i7,10th,16 GB GB,DDR4,256 GB,1024 GB,Windows,...,1,No,No,119990,140990,14,4.6,2038,313,19.669009


In [80]:
df.loc[df.sort_values(by='score', ascending=False).head(10).index[0]]

brand                  realme
model                    Book
processor_brand         Intel
processor_name        Core i3
processor_gnrtn          11th
ram_gb                8 GB GB
ram_type                 DDR4
ssd                    256 GB
hdd                      0 GB
os                    Windows
os_bit                 64-bit
graphic_card_gb             0
weight             ThinNlight
display_size               14
warranty                    1
Touchscreen                No
msoffice                  Yes
latest_price            46990
old_price               54999
discount                   14
star_rating               4.4
ratings                  6352
reviews                  1014
score               22.016466
Name: 44, dtype: object

In [81]:
model = cp_model.CpModel()
var_upper_bound = 1000
x1 = model.NewIntVar(0, var_upper_bound, 'x1')
x2 = model.NewIntVar(0, var_upper_bound, 'x2')
x3 = model.NewIntVar(0, 0, 'x3')
x4 = model.NewIntVar(0, 0, 'x4')
x5 = model.NewIntVar(0, 0, 'x5')
x6 = model.NewIntVar(0, 0, 'x6')

# Обмеження
model.Add(3 * x1 + 4 * x2 - 2 * x4 <= 24)
model.Add(x1 + 2 * x2 - x3 <= 8)
model.Add(4 * x1 - x5 <= 16)
model.Add(4 * x2 - x6 <= 12)

# Цільова функція ефективності
efficiency_function = - 2 * x1 - 2 * x2

# model.Maximize(efficiency_function) # Максимізоція цільової функції
model.Minimize(efficiency_function)  # Мінімізація цільової функції

# Вирішувач
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('---------------Optimal efficiency---------------------')
    print('Minimum of objective function: %i' % solver.ObjectiveValue())
    x1out = solver.Value(x1)
    x2out = solver.Value(x2)
    print('x1= ', x1out)
    print('x2= ', x2out)



    print('---------------- Checked maximum ------------------------')
    x1max = 4
    x2max = 2
    MaximusP = - 2 * x1max - 2 * x2max
    print('MaximusP= ', MaximusP)


---------------Optimal efficiency---------------------
Minimum of objective function: -12
x1=  4
x2=  2
---------------- Checked maximum ------------------------
MaximusP=  -12
